In [1]:
import os, sys
sys.path.append(os.path.join(os.path.abspath(''), '../../'))

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
from palettable.scientific.sequential import Oslo_5_r

from misc.normalization import (
    LogNormalizer,
    DummyNormalizer
)
from misc.Directory import getVkPredictRootDir
from scipy.stats import spearmanr
from misc.metric import compute_metrics

In [2]:
methods = {
    "SH": "_base",
    "PILR": "_pilr",
    "NN": "",
    "SH-Const": "_const_base",
    "PILR-Const": "_const_pilr",
    "NN-Const": "_const_fixed",
}

names = {
    '3060': 'RTX3060',
    '630': 'UHD630',
    '4060': 'RTX4060',
    '1660': 'GTX1660Ti',
    '7900': 'RX7900GRE'
}

measured = {k: {} for k in methods.keys()}
predicted = {k: {} for k in methods.keys()}

for name in names.keys():
    for method in methods:
        if method == "NN" or method == "NN-Const":
            measured[method][name] = LogNormalizer().invNormalize(np.load(os.path.join(getVkPredictRootDir(), f'validation/{name}{methods[method]}_labels.npy')))
            predicted[method][name] = LogNormalizer().invNormalize(np.load(os.path.join(getVkPredictRootDir(), f'validation/{name}{methods[method]}_preds.npy')))
        else:
            measured[method][name] = np.load(os.path.join(getVkPredictRootDir(), f'validation/{name}{methods[method]}_labels.npy'))
            predicted[method][name] = np.load(os.path.join(getVkPredictRootDir(), f'validation/{name}{methods[method]}_preds.npy'))


In [3]:
ss = "Platform\t"
for method in methods:
    ss += f"{method}-mape\t{method}-sprm\t"
print(ss)

for i, archName in enumerate(names.keys()):
    ss = f"{names[archName]:8}\t"
    for method in methods:
        mape = compute_metrics(predicted[method][archName], measured[method][archName])['mape']
        corr, _ = spearmanr(measured[method][archName], predicted[method][archName])
        ss += f"{mape:.4}\t{corr:.4}\t"
    
    print(ss)


Platform	SH-mape	SH-sprm	PILR-mape	PILR-sprm	NN-mape	NN-sprm	SH-Const-mape	SH-Const-sprm	PILR-Const-mape	PILR-Const-sprm	NN-Const-mape	NN-Const-sprm	
RTX3060 	0.7526	0.9584	0.469	0.92	0.4044	0.9642	0.7602	0.7131	0.8643	0.8551	0.8102	0.8608	


c:\Projects\NGPP\vkPredict\notebooks\production\../..\misc\metric.py:39: RuntimeWarning: Mean of empty slice.
  ((candRealTime - candPredTime)**2).mean(axis=0)
c:\Projects\NGPP\venv\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Projects\NGPP\venv\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


UHD630  	0.3073	0.9594	0.2732	0.9676	0.266	0.9722	0.6697	0.7152	0.7287	0.8503	0.7042	0.8749	
RTX4060 	0.8117	0.9588	0.556	0.929	0.4462	0.9632	0.7954	0.7118	0.9039	0.8563	0.981	0.7518	
GTX1660Ti	0.8256	0.9607	0.5499	0.9353	0.4159	0.9681	0.802	0.7142	0.9026	0.8571	1.096	0.8784	
RX7900GRE	0.3632	0.951	0.3628	0.9433	0.2654	0.9578	0.6865	0.6857	0.8119	0.8388	0.5116	0.8448	


In [4]:
# for method in ['SH', 'PILR', 'NN']:
#     for i, archName in enumerate(names.keys()):
#         # corrNN, _ = pearsonr(measured[archName], predicted[archName])
#         # corrPILR, _ = pearsonr(measured_base[archName], predicted_base[archName])
#         # corrSH, _ = pearsonr(measured_pilr[archName], predicted_pilr[archName])
#         # print(f"{archName} Pearson: corrNN={corrNN:.4f}, corrPILR={corrPILR:.4f} ({'NN' if corrNN > corrPILR else 'PILR'} better)")

#         pre = None
#         if method == "SH":
#             pre = predicted[archName]
#         elif method == "PILR":
#             pre = predicted_pilr[archName]
            
#         mapeNN = compute_metrics(predicted[archName], measured[archName])['mape']
#         corrNN, _ = spearmanr(measured[archName], predicted[archName])
#         mapeNNAvg += mapeNN

#         mapeSH = compute_metrics(predicted_base[archName], measured_base[archName])['mape']
#         corrSH, _ = spearmanr(measured_base[archName], predicted_base[archName])
#         mapeSHAvg += mapeSH

#         mapePILR = compute_metrics(predicted_pilr[archName], measured_pilr[archName])['mape']
#         corrPILR, _ = spearmanr(measured_pilr[archName], predicted_pilr[archName])
#         mapePILRAvg += mapePILR

#         spearmanBig = max(corrNN, corrPILR, corrSH)
#         mapeSmall = min(mapeNN, mapePILR, mapeSH)

        

KeyError: '3060'